Deep Learning
=============

Assignment 4
------------

Previously in `2_fullyconnected.ipynb` and `3_regularization.ipynb`, we trained fully connected networks to classify [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) characters.

The goal of this assignment is make the neural network convolutional.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a TensorFlow-friendly shape:
- convolutions need the image data formatted as a cube (width by height by #channels)
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

import numpy as np

def reformat(dataset, labels):
  dataset = dataset.reshape(
    (-1, image_size, image_size, num_channels)).astype(np.float32)
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28, 1) (200000, 10)
Validation set (10000, 28, 28, 1) (10000, 10)
Test set (10000, 28, 28, 1) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

Let's build a small network with two convolutional layers, followed by one fully connected layer. Convolutional networks are more expensive computationally, so we'll limit its depth and number of fully connected nodes.

In [5]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [7]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3.213093
Minibatch accuracy: 18.8%
Validation accuracy: 10.0%
Minibatch loss at step 50: 1.705882
Minibatch accuracy: 25.0%
Validation accuracy: 44.3%
Minibatch loss at step 100: 1.193937
Minibatch accuracy: 50.0%
Validation accuracy: 62.9%
Minibatch loss at step 150: 0.464993
Minibatch accuracy: 93.8%
Validation accuracy: 72.7%
Minibatch loss at step 200: 0.703133
Minibatch accuracy: 81.2%
Validation accuracy: 77.4%
Minibatch loss at step 250: 1.254208
Minibatch accuracy: 68.8%
Validation accuracy: 77.5%
Minibatch loss at step 300: 0.312940
Minibatch accuracy: 93.8%
Validation accuracy: 79.2%
Minibatch loss at step 350: 0.552538
Minibatch accuracy: 93.8%
Validation accuracy: 76.3%
Minibatch loss at step 400: 0.359237
Minibatch accuracy: 93.8%
Validation accuracy: 80.2%
Minibatch loss at step 450: 0.724704
Minibatch accuracy: 87.5%
Validation accuracy: 78.5%
Minibatch loss at step 500: 0.640203
Minibatch accuracy: 87.5%
Validation accuracy: 80.4%
M

---
Problem 1
---------

The convolutional model above uses convolutions with stride 2 to reduce the dimensionality. Replace the strides by a max pooling operation (`nn.max_pool()`) of stride 2 and kernel size 2.

---

In [8]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv1 = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')
    bias1 = tf.nn.relu(conv1 + layer1_biases)
    pool1 = tf.nn.max_pool(bias1, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
    conv2 = tf.nn.conv2d(pool1, layer2_weights, [1, 1, 1, 1], padding='SAME')
    bias2 = tf.nn.relu(conv2 + layer2_biases)
    pool2 = tf.nn.max_pool(bias2, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
    shape = pool2.get_shape().as_list()
    reshape = tf.reshape(pool2, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [9]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3.779494
Minibatch accuracy: 6.2%
Validation accuracy: 10.0%
Minibatch loss at step 50: 1.866151
Minibatch accuracy: 37.5%
Validation accuracy: 40.1%
Minibatch loss at step 100: 1.125046
Minibatch accuracy: 56.2%
Validation accuracy: 56.2%
Minibatch loss at step 150: 0.518518
Minibatch accuracy: 87.5%
Validation accuracy: 75.0%
Minibatch loss at step 200: 1.233413
Minibatch accuracy: 50.0%
Validation accuracy: 76.1%
Minibatch loss at step 250: 1.081354
Minibatch accuracy: 68.8%
Validation accuracy: 77.6%
Minibatch loss at step 300: 0.417454
Minibatch accuracy: 87.5%
Validation accuracy: 79.8%
Minibatch loss at step 350: 0.428833
Minibatch accuracy: 93.8%
Validation accuracy: 79.4%
Minibatch loss at step 400: 0.205477
Minibatch accuracy: 100.0%
Validation accuracy: 81.0%
Minibatch loss at step 450: 0.671585
Minibatch accuracy: 87.5%
Validation accuracy: 79.6%
Minibatch loss at step 500: 0.673592
Minibatch accuracy: 87.5%
Validation accuracy: 81.7%
M

---
Problem 2
---------

Try to get the best performance you can using a convolutional net. Look for example at the classic [LeNet5](http://yann.lecun.com/exdb/lenet/) architecture, adding Dropout, and/or adding learning rate decay.

---

The CNN below is loosely inspired by the LeNet5 architecture.

In [10]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  size3 = ((image_size - patch_size + 1) // 2 - patch_size + 1) // 2
  layer3_weights = tf.Variable(tf.truncated_normal(
      [size3 * size3 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    # C1 input 28 x 28
    conv1 = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='VALID')
    bias1 = tf.nn.relu(conv1 + layer1_biases)
    # S2 input 24 x 24
    pool2 = tf.nn.avg_pool(bias1, [1, 2, 2, 1], [1, 2, 2, 1], padding='VALID')
    # C3 input 12 x 12
    conv3 = tf.nn.conv2d(pool2, layer2_weights, [1, 1, 1, 1], padding='VALID')
    bias3 = tf.nn.relu(conv3 + layer2_biases)
    # S4 input 8 x 8
    pool4 = tf.nn.avg_pool(bias3, [1, 2, 2, 1], [1, 2, 2, 1], padding='VALID')
    # F6 input 4 x 4
    shape = pool4.get_shape().as_list()
    reshape = tf.reshape(pool4, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [11]:
num_steps = 20001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.887526
Minibatch accuracy: 6.2%
Validation accuracy: 10.0%
Minibatch loss at step 50: 1.755834
Minibatch accuracy: 43.8%
Validation accuracy: 49.1%
Minibatch loss at step 100: 1.122573
Minibatch accuracy: 62.5%
Validation accuracy: 59.9%
Minibatch loss at step 150: 0.818305
Minibatch accuracy: 81.2%
Validation accuracy: 68.2%
Minibatch loss at step 200: 1.141408
Minibatch accuracy: 62.5%
Validation accuracy: 70.2%
Minibatch loss at step 250: 1.179383
Minibatch accuracy: 75.0%
Validation accuracy: 74.6%
Minibatch loss at step 300: 0.573698
Minibatch accuracy: 87.5%
Validation accuracy: 75.9%
Minibatch loss at step 350: 0.567827
Minibatch accuracy: 81.2%
Validation accuracy: 73.7%
Minibatch loss at step 400: 0.373525
Minibatch accuracy: 100.0%
Validation accuracy: 78.5%
Minibatch loss at step 450: 0.923118
Minibatch accuracy: 81.2%
Validation accuracy: 77.4%
Minibatch loss at step 500: 0.829145
Minibatch accuracy: 87.5%
Validation accuracy: 78.0%
M

Validation accuracy: 86.8%
Minibatch loss at step 4550: 0.372265
Minibatch accuracy: 87.5%
Validation accuracy: 87.1%
Minibatch loss at step 4600: 0.533304
Minibatch accuracy: 87.5%
Validation accuracy: 86.2%
Minibatch loss at step 4650: 0.773061
Minibatch accuracy: 81.2%
Validation accuracy: 86.3%
Minibatch loss at step 4700: 0.506120
Minibatch accuracy: 87.5%
Validation accuracy: 87.0%
Minibatch loss at step 4750: 0.928353
Minibatch accuracy: 68.8%
Validation accuracy: 86.3%
Minibatch loss at step 4800: 0.489047
Minibatch accuracy: 87.5%
Validation accuracy: 86.7%
Minibatch loss at step 4850: 0.354044
Minibatch accuracy: 93.8%
Validation accuracy: 87.1%
Minibatch loss at step 4900: 0.119216
Minibatch accuracy: 93.8%
Validation accuracy: 87.0%
Minibatch loss at step 4950: 0.178830
Minibatch accuracy: 93.8%
Validation accuracy: 87.2%
Minibatch loss at step 5000: 0.955608
Minibatch accuracy: 68.8%
Validation accuracy: 86.2%
Minibatch loss at step 5050: 0.237759
Minibatch accuracy: 93.8%

Validation accuracy: 88.1%
Minibatch loss at step 9050: 0.340914
Minibatch accuracy: 87.5%
Validation accuracy: 88.0%
Minibatch loss at step 9100: 0.406798
Minibatch accuracy: 87.5%
Validation accuracy: 87.7%
Minibatch loss at step 9150: 0.764048
Minibatch accuracy: 75.0%
Validation accuracy: 87.2%
Minibatch loss at step 9200: 0.293021
Minibatch accuracy: 87.5%
Validation accuracy: 88.0%
Minibatch loss at step 9250: 0.839284
Minibatch accuracy: 75.0%
Validation accuracy: 87.9%
Minibatch loss at step 9300: 0.926702
Minibatch accuracy: 81.2%
Validation accuracy: 87.5%
Minibatch loss at step 9350: 0.294565
Minibatch accuracy: 87.5%
Validation accuracy: 87.4%
Minibatch loss at step 9400: 0.348141
Minibatch accuracy: 87.5%
Validation accuracy: 87.9%
Minibatch loss at step 9450: 0.339952
Minibatch accuracy: 93.8%
Validation accuracy: 87.9%
Minibatch loss at step 9500: 0.244894
Minibatch accuracy: 93.8%
Validation accuracy: 87.9%
Minibatch loss at step 9550: 0.226825
Minibatch accuracy: 100.0

Minibatch loss at step 13500: 0.388597
Minibatch accuracy: 87.5%
Validation accuracy: 88.2%
Minibatch loss at step 13550: 0.395972
Minibatch accuracy: 93.8%
Validation accuracy: 88.2%
Minibatch loss at step 13600: 0.326045
Minibatch accuracy: 87.5%
Validation accuracy: 88.6%
Minibatch loss at step 13650: 0.497251
Minibatch accuracy: 87.5%
Validation accuracy: 88.4%
Minibatch loss at step 13700: 0.386662
Minibatch accuracy: 81.2%
Validation accuracy: 88.5%
Minibatch loss at step 13750: 0.576945
Minibatch accuracy: 81.2%
Validation accuracy: 88.6%
Minibatch loss at step 13800: 0.047079
Minibatch accuracy: 100.0%
Validation accuracy: 88.0%
Minibatch loss at step 13850: 0.130557
Minibatch accuracy: 93.8%
Validation accuracy: 88.1%
Minibatch loss at step 13900: 0.168503
Minibatch accuracy: 93.8%
Validation accuracy: 88.5%
Minibatch loss at step 13950: 0.453997
Minibatch accuracy: 81.2%
Validation accuracy: 88.5%
Minibatch loss at step 14000: 0.054875
Minibatch accuracy: 100.0%
Validation ac

Minibatch loss at step 17950: 0.356499
Minibatch accuracy: 87.5%
Validation accuracy: 88.8%
Minibatch loss at step 18000: 0.334625
Minibatch accuracy: 81.2%
Validation accuracy: 88.2%
Minibatch loss at step 18050: 0.343583
Minibatch accuracy: 87.5%
Validation accuracy: 88.8%
Minibatch loss at step 18100: 0.647946
Minibatch accuracy: 87.5%
Validation accuracy: 89.2%
Minibatch loss at step 18150: 0.221066
Minibatch accuracy: 87.5%
Validation accuracy: 89.1%
Minibatch loss at step 18200: 0.406646
Minibatch accuracy: 93.8%
Validation accuracy: 89.0%
Minibatch loss at step 18250: 0.407224
Minibatch accuracy: 81.2%
Validation accuracy: 89.1%
Minibatch loss at step 18300: 0.115090
Minibatch accuracy: 100.0%
Validation accuracy: 88.9%
Minibatch loss at step 18350: 0.321624
Minibatch accuracy: 87.5%
Validation accuracy: 89.0%
Minibatch loss at step 18400: 0.731389
Minibatch accuracy: 75.0%
Validation accuracy: 88.5%
Minibatch loss at step 18450: 0.288471
Minibatch accuracy: 93.8%
Validation acc

The accuracy is good, but not as good as the 3-layer network from the previous assignment.

The next version of the net uses dropout and learning rate decay:

In [13]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64
beta_regul = 1e-3
drop_out = 0.5

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  global_step = tf.Variable(0)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  size3 = ((image_size - patch_size + 1) // 2 - patch_size + 1) // 2
  layer3_weights = tf.Variable(tf.truncated_normal(
      [size3 * size3 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_hidden], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer5_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer5_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data, keep_prob):
    # C1 input 28 x 28
    conv1 = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='VALID')
    bias1 = tf.nn.relu(conv1 + layer1_biases)
    # S2 input 24 x 24
    pool2 = tf.nn.avg_pool(bias1, [1, 2, 2, 1], [1, 2, 2, 1], padding='VALID')
    # C3 input 12 x 12
    conv3 = tf.nn.conv2d(pool2, layer2_weights, [1, 1, 1, 1], padding='VALID')
    bias3 = tf.nn.relu(conv3 + layer2_biases)
    # S4 input 8 x 8
    pool4 = tf.nn.avg_pool(bias3, [1, 2, 2, 1], [1, 2, 2, 1], padding='VALID')
    # F5 input 4 x 4
    shape = pool4.get_shape().as_list()
    reshape = tf.reshape(pool4, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden5 = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    # F6
    drop5 = tf.nn.dropout(hidden5, keep_prob)
    hidden6 = tf.nn.relu(tf.matmul(hidden5, layer4_weights) + layer4_biases)
    drop6 = tf.nn.dropout(hidden6, keep_prob)
    return tf.matmul(drop6, layer5_weights) + layer5_biases
  
  # Training computation.
  logits = model(tf_train_dataset, drop_out)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
  # Optimizer.
  learning_rate = tf.train.exponential_decay(0.05, global_step, 1000, 0.85, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset, 1.0))
  test_prediction = tf.nn.softmax(model(tf_test_dataset, 1.0))

In [14]:
num_steps = 5001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.466877
Minibatch accuracy: 25.0%
Validation accuracy: 10.3%
Minibatch loss at step 50: 2.161342
Minibatch accuracy: 18.8%
Validation accuracy: 26.2%
Minibatch loss at step 100: 1.929306
Minibatch accuracy: 37.5%
Validation accuracy: 44.5%
Minibatch loss at step 150: 1.402269
Minibatch accuracy: 43.8%
Validation accuracy: 56.8%
Minibatch loss at step 200: 1.576180
Minibatch accuracy: 43.8%
Validation accuracy: 59.8%
Minibatch loss at step 250: 1.603382
Minibatch accuracy: 56.2%
Validation accuracy: 67.5%
Minibatch loss at step 300: 0.996797
Minibatch accuracy: 75.0%
Validation accuracy: 74.0%
Minibatch loss at step 350: 0.924196
Minibatch accuracy: 81.2%
Validation accuracy: 72.1%
Minibatch loss at step 400: 0.585693
Minibatch accuracy: 75.0%
Validation accuracy: 72.5%
Minibatch loss at step 450: 1.226259
Minibatch accuracy: 75.0%
Validation accuracy: 74.3%
Minibatch loss at step 500: 1.268786
Minibatch accuracy: 75.0%
Validation accuracy: 75.3%
M

Validation accuracy: 85.3%
Minibatch loss at step 4550: 0.539713
Minibatch accuracy: 81.2%
Validation accuracy: 85.3%
Minibatch loss at step 4600: 0.340253
Minibatch accuracy: 87.5%
Validation accuracy: 85.7%
Minibatch loss at step 4650: 1.693758
Minibatch accuracy: 81.2%
Validation accuracy: 85.5%
Minibatch loss at step 4700: 0.515037
Minibatch accuracy: 81.2%
Validation accuracy: 85.4%
Minibatch loss at step 4750: 1.176289
Minibatch accuracy: 62.5%
Validation accuracy: 85.6%
Minibatch loss at step 4800: 0.575342
Minibatch accuracy: 81.2%
Validation accuracy: 85.6%
Minibatch loss at step 4850: 0.411686
Minibatch accuracy: 87.5%
Validation accuracy: 85.7%
Minibatch loss at step 4900: 0.197935
Minibatch accuracy: 93.8%
Validation accuracy: 85.7%
Minibatch loss at step 4950: 0.303288
Minibatch accuracy: 87.5%
Validation accuracy: 85.8%
Minibatch loss at step 5000: 1.693749
Minibatch accuracy: 68.8%
Validation accuracy: 85.5%
Test accuracy: 91.9%


Well, the accuracy is worst. This net has many meta parameters and I don't feel comfortable in tuning them randomly. I should probably change the depth and make it different between the layers, since it looks like the increasing number of feature maps is a key design item.